# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
import fasttext
ft_path = 'lid.176.bin' # path to fasttext model
import re

In [2]:
# load messages dataset
messages = pd.read_csv('data/disaster_messages.csv')
display(messages.head())
display(messages['message'][0])

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


'Weather update - a cold front from Cuba that could pass over Haiti'

In [3]:
# load categories dataset
categories = pd.read_csv('data/disaster_categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [4]:
# merge datasets
df = messages.merge(categories, on='id')
display(df.head())
display(df['message'][0])

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


'Weather update - a cold front from Cuba that could pass over Haiti'

### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [5]:
# create a dataframe of the 36 individual category columns
categories = df['categories'].str.split(';', expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [6]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26386 entries, 0 to 26385
Data columns (total 36 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       26386 non-null  object
 1   1       26386 non-null  object
 2   2       26386 non-null  object
 3   3       26386 non-null  object
 4   4       26386 non-null  object
 5   5       26386 non-null  object
 6   6       26386 non-null  object
 7   7       26386 non-null  object
 8   8       26386 non-null  object
 9   9       26386 non-null  object
 10  10      26386 non-null  object
 11  11      26386 non-null  object
 12  12      26386 non-null  object
 13  13      26386 non-null  object
 14  14      26386 non-null  object
 15  15      26386 non-null  object
 16  16      26386 non-null  object
 17  17      26386 non-null  object
 18  18      26386 non-null  object
 19  19      26386 non-null  object
 20  20      26386 non-null  object
 21  21      26386 non-null  object
 22  22      26386 non-null

In [7]:
categories.nunique()

0     3
1     2
2     2
3     2
4     2
5     2
6     2
7     2
8     2
9     1
10    2
11    2
12    2
13    2
14    2
15    2
16    2
17    2
18    2
19    2
20    2
21    2
22    2
23    2
24    2
25    2
26    2
27    2
28    2
29    2
30    2
31    2
32    2
33    2
34    2
35    2
dtype: int64

In [8]:
categories.iloc[:,0].unique()

array(['related-1', 'related-0', 'related-2'], dtype=object)

In [9]:
categories.iloc[:,9].unique()

array(['child_alone-0'], dtype=object)

In [10]:
# select the first row of the categories dataframe
row = categories.iloc[0,:]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.str.rstrip('-01')
print(category_colnames)

0                    related
1                    request
2                      offer
3                aid_related
4               medical_help
5           medical_products
6          search_and_rescue
7                   security
8                   military
9                child_alone
10                     water
11                      food
12                   shelter
13                  clothing
14                     money
15            missing_people
16                  refugees
17                     death
18                 other_aid
19    infrastructure_related
20                 transport
21                 buildings
22               electricity
23                     tools
24                 hospitals
25                     shops
26               aid_centers
27      other_infrastructure
28           weather_related
29                    floods
30                     storm
31                      fire
32                earthquake
33                      cold
34            

In [11]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [12]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories.loc[:,column].str[-1]
    
    # convert column from string to numeric
    categories[column] = categories[column].astype('int64')
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [13]:
# drop the original categories column from `df`
df.drop(columns=['categories'], inplace=True)

df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [14]:
display(df['message'][0])

'Weather update - a cold front from Cuba that could pass over Haiti'

In [15]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
display(df['message'][0])

'Weather update - a cold front from Cuba that could pass over Haiti'

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [17]:
# check number of duplicates
df.duplicated().sum()

170

In [18]:
# drop duplicates
df.drop_duplicates(inplace=True, ignore_index=True)

In [19]:
# check number of duplicates
df.duplicated().sum()

0

In [20]:
# this seems to be an outlier r/t to the other categories
df[df['related']==2].describe().iloc[:,0:15]

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing
count,188.000000,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0
mean,11703.340426,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,5479.507080,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,146.000000,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,8956.000000,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,13770.000000,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,14376.750000,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,29126.000000,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
df[df['related']==2].describe().iloc[:,15:30]

,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related
count,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0,188.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
df[df['related']==2].describe().iloc[:,30:]

,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,188.0,188.0,188.0,188.0,188.0,188.0,188.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
df[df['related']==0].describe().iloc[:,0:15]

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing
count,6122.000000,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0
mean,13819.700751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,8843.881743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,14.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,6731.750000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,10467.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,22588.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,30262.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
df[df['related']==0].describe().iloc[:,15:30]

,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related
count,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
df[df['related']==0].describe().iloc[:,30:]

,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0,6122.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
df[df['related']==1].describe().iloc[:,0:15]

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing
count,19906.000000,19906.0,19906.000000,19906.000000,19906.000000,19906.000000,19906.000000,19906.000000,19906.000000,19906.000000,19906.0,19906.000000,19906.000000,19906.000000,19906.000000
mean,15690.218025,1.0,0.224756,0.005928,0.545564,0.104692,0.065960,0.036371,0.023661,0.043203,0.0,0.083995,0.146840,0.116246,0.020346
std,8794.225162,0.0,0.417432,0.076766,0.497932,0.306164,0.248218,0.187216,0.151995,0.203319,0.0,0.277387,0.353955,0.320528,0.141183
min,2.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,7896.250000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,16595.000000,1.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
75%,23058.750000,1.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
max,30265.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000


It looks like both the `related = 0` and `related=2` have no other categories selected.  
See if any difference in messages.

In [27]:
df[df['related']==2][['message','original','genre']]

,message,original,genre
117,Dans la zone de Saint Etienne la route de Jacm...,Nan zon st. etine rout jakmel la bloke se mize...,direct
219,. .. i with limited means. Certain patients co...,t avec des moyens limites. Certains patients v...,direct
305,The internet caf Net@le that's by the Dal road...,Cyber cafe net@le ki chita rout de dal tou pr ...,direct
460,"Bonsoir, on est a bon repos aprs la compagnie ...",Bonswa nou nan bon repo apri teleko nan wout t...,direct
576,URGENT CRECHE ORPHANAGE KAY TOUT TIMOUN CROIX ...,r et Salon Furterer. mwen se yon Cosmtologue. ...,direct
...,...,...,...
20324,transferred to a sanitary landfill site by a,NaN,news
20495,Families also have solar lamps which can be re...,NaN,news
22326,Read the [full blog post](http://www.odi.org.u...,NaN,news
23382,Actualmente e independientemente de la ayuda d...,NaN,news


In [28]:
df[df['related']==0][['message','original','genre']]

,message,original,genre
5,Information about the National Palace-,Informtion au nivaux palais nationl,direct
8,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct
11,I am in Petionville. I need more information r...,M nan pv mvle plis enfomasyon sou 4636,direct
17,are you going to call me or do you want me to ...,Eske se rele nap relem oubyen se mwen kap rele...,direct
18,I don't understand how to use this thing 4636.,Mwen pa konprann koman pou m itilize bagay 463...,direct
...,...,...,...
26200,WHO is recruiting a sanitary engineer / consul...,NaN,news
26204,"Cadmium, a metallic element widely used in bat...",NaN,news
26209,However while ECOWAS wanted him to lead a 12-m...,NaN,news
26211,The training demonstrated how to enhance micro...,NaN,news


In [29]:
df[df['related']==1][['message','original','genre']]

,message,original,genre
0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct
...,...,...,...
26208,The delivery was made in conjunction with the ...,NaN,news
26210,"Hpakant, an area rich with coveted jade stones...",NaN,news
26213,"Proshika, operating in Cox's Bazar municipalit...",NaN,news
26214,"Some 2,000 women protesting against the conduc...",NaN,news


It looks like the zero & two "relateds" have nothing in the other categories. My guess is "related" is code something like no = 0, yes = 1, and (maybe) maybe =2. 

In [30]:
display(df['message'][0])
display(df['message'][5])
display(df['message'][20495])
display(df['message'][26215])
display(df['message'][26211])
display(df['message'][25215])
display(df['message'][26212])
display(df['message'][26200])
display(df['message'][219])
display(df['message'][305])

'Weather update - a cold front from Cuba that could pass over Haiti'

'Information about the National Palace-'

'Families also have solar lamps which can be recharged at charging stations in each community.'

'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'

'The training demonstrated how to enhance micronutrient, protein and energy intake for meals using locally grown horticulture produce, rice and fish.'

'Mali\'s former President Amadou Toumani Touré - though hailed internationally as a democrat, reformer, and ally in the American "war on terror"  - oversaw a system tainted by corruption at all levels.'

'A suitable candidate has been selected and OCHA Jakarta is currently working on her three month-contract starting on the 01st July 2005.'

'WHO is recruiting a sanitary engineer / consultant from 17 February to 31 March with following Terms of References:'

'. .. i with limited means. Certain patients come from the capital.'

"The internet caf Net@le that's by the Dal road by the Maranata church ( incomplete )"

It's not entirely clear what `related = 2` means. It does look like they might be ambiguous messages that were meant to be reclassified later. Since we don't have any easy way of relabeling these and since they're a small part of the dataset (N = 188 out of 26k), we'll drop them

In [31]:
# NB here the first time I did it without the first set of parens and it wiped the whole df
df[~(df['related']==2)]['related'].unique()

array([1, 0])

In [32]:
df = df[~(df['related']==2)]
display(df['related'].unique())
display(df.head())

array([1, 0])

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
display(df.shape)

(26028, 40)

Also, the `child_alone` category was not associated with any messages, so we won't be able to use this with a classifier, as there's only one category. We'll drop it.

In [36]:
display(df['child_alone'].unique())

array([0])

In [38]:
df.drop(columns=['child_alone'], inplace=True)

Some of the messages previewed above appear not to be in English. These may have just been the messages that were in related=2 and are now gone.  
But, to be sure, will run [fastText](https://fasttext.cc/blog/2017/10/02/blog-post.html) language id on the messages to see if any aren't identified as English.

In [39]:
ft_model=fasttext.load_model(ft_path)
# note this gives a tuple, tho b/c it said "object" I assumed it was a string, til the replace stuff below failed
# so, coercing to string. the 1st [0] subscript drops the probability element, the 2nd [0] extracts the label text
df['pred_lang']=df['message'].apply(ft_model.predict).str[0].str[0]

In [40]:
df['pred_lang'].head()

0    __label__en
1    __label__en
2    __label__en
3    __label__en
4    __label__en
Name: pred_lang, dtype: object

In [41]:
# simplify this field a bit
df['pred_lang'] = df['pred_lang'].str.replace("__label__",'',regex=False)
display(df['pred_lang'].head())

0    en
1    en
2    en
3    en
4    en
Name: pred_lang, dtype: object

In [42]:
df['pred_lang'].unique()

array(['en', 'ja', 'fr', 'es', 'zh', 'de', 'gl', 'it', 'la', 'pt', 'nl'],
      dtype=object)

In [43]:
# it looks like there are still some non-Englis entries
df['pred_lang'].value_counts()

en    25900
ja       59
es       24
fr       20
pt       10
de        4
it        4
zh        3
nl        2
gl        1
la        1
Name: pred_lang, dtype: int64

In [44]:
# drop the non-english entries, since it doesn't make sense to try to run text identification 
# using an english dictionary on non-english phrases
df = df[df['pred_lang']=='en']
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,pred_lang
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,en
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,en
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,en


In [45]:
# check it
df['pred_lang'].value_counts()

en    25900
Name: pred_lang, dtype: int64

In [46]:
# we can drop pred_lang now
df.drop(columns=['pred_lang'], inplace=True)
df.head()

/home/umahp/data_science/dsenv/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [47]:
display(df.head(1))
display([df.columns[4:]])

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


[Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
        'medical_products', 'search_and_rescue', 'security', 'military',
        'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
        'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
        'buildings', 'electricity', 'tools', 'hospitals', 'shops',
        'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
        'storm', 'fire', 'earthquake', 'cold', 'other_weather',
        'direct_report'],
       dtype='object')]

In [48]:
sql_start = "CREATE TABLE messages (id INTEGER PRIMARY KEY , message TEXT, original TEXT, genre TEXT, "
display(sql_start)
cols = df.columns[4:].to_list()
all_cols = ' INTEGER, '.join(cols)
display(all_cols)
sql_create = sql_start + all_cols + ' INTEGER);'
display(sql_create)
    

'CREATE TABLE messages (id INTEGER PRIMARY KEY , message TEXT, original TEXT, genre TEXT, '

'related INTEGER, request INTEGER, offer INTEGER, aid_related INTEGER, medical_help INTEGER, medical_products INTEGER, search_and_rescue INTEGER, security INTEGER, military INTEGER, water INTEGER, food INTEGER, shelter INTEGER, clothing INTEGER, money INTEGER, missing_people INTEGER, refugees INTEGER, death INTEGER, other_aid INTEGER, infrastructure_related INTEGER, transport INTEGER, buildings INTEGER, electricity INTEGER, tools INTEGER, hospitals INTEGER, shops INTEGER, aid_centers INTEGER, other_infrastructure INTEGER, weather_related INTEGER, floods INTEGER, storm INTEGER, fire INTEGER, earthquake INTEGER, cold INTEGER, other_weather INTEGER, direct_report'

'CREATE TABLE messages (id INTEGER PRIMARY KEY , message TEXT, original TEXT, genre TEXT, related INTEGER, request INTEGER, offer INTEGER, aid_related INTEGER, medical_help INTEGER, medical_products INTEGER, search_and_rescue INTEGER, security INTEGER, military INTEGER, water INTEGER, food INTEGER, shelter INTEGER, clothing INTEGER, money INTEGER, missing_people INTEGER, refugees INTEGER, death INTEGER, other_aid INTEGER, infrastructure_related INTEGER, transport INTEGER, buildings INTEGER, electricity INTEGER, tools INTEGER, hospitals INTEGER, shops INTEGER, aid_centers INTEGER, other_infrastructure INTEGER, weather_related INTEGER, floods INTEGER, storm INTEGER, fire INTEGER, earthquake INTEGER, cold INTEGER, other_weather INTEGER, direct_report INTEGER);'

In [49]:
# trying to use id as primary key failed, so may not be unique
display(df.shape)
display(df['id'].nunique())

(25900, 39)

25865

In [50]:
display(df.duplicated(subset='id').sum())

35

In [51]:
display(df[df.duplicated(subset='id', keep=False)==True])

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
162,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
163,202,?? port au prince ?? and food. they need gover...,p bay pap la syen ak manje. Yo bezwen ed gouve...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
708,862,What is the address of the radio station? I as...,Ki adres radyo a? Paske m bezwen al depoze dos...,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
709,862,What is the address of the radio station? I as...,Ki adres radyo a? Paske m bezwen al depoze dos...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1406,1652,"please we need water, food and tents, we have ...","p jwen dlo, manje, tant pou nou demi nou gen 1...",direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24010,27768,An EU-backed French offensive against rebels r...,NaN,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24617,28462,"To date, a total of 39 people suffering from E...",NaN,news,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
24618,28462,"To date, a total of 39 people suffering from E...",NaN,news,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
25124,29022,"In a field in Jallouzai, just inside Pakistan,...",NaN,news,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


The duplicated ids appear to have identical content, but different categories. Will assume the 2nd message for each is "updated", since the ones reviewed appear to have additional categories added.

In [52]:
# drop the duplicate-id entries, keeping the last one
df.drop_duplicates(subset='id', inplace=True, ignore_index=True, keep='last')
# check number of duplicates
df.duplicated(subset='id').sum()

<ipython-input-52-b7f471e0eaa9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset='id', inplace=True, ignore_index=True, keep='last')


0

In [53]:
#engine = create_engine('sqlite:///InsertDatabaseName.db')
#df.to_sql('InsertTableName', engine, index=False)
# gonna just do this with sqlite3 since have the code available
# this will create DB if doesn't exist
conn = sqlite3.connect('TestETL.db')

# get a cursor
cur = conn.cursor()

# drop the messages table in case it already exists
cur.execute("DROP TABLE IF EXISTS messages")


# create the messages table
cur.execute(sql_create)

# write the df
df.to_sql('messages', conn, index=False, if_exists='append')

# check it
cur.execute("SELECT * FROM messages")
cur.fetchmany(10)


[(2,
  'Weather update - a cold front from Cuba that could pass over Haiti',
  'Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi',
  'direct',
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0),
 (7,
  'Is the Hurricane over or is it not over',
  'Cyclone nan fini osinon li pa fini',
  'direct',
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0),
 (8,
  'Looking for someone but no name',
  'Patnm, di Maryani relem pou li banm nouvel li ak timoun yo. Mesi se john jean depi Monben kwochi.',
  'direct',
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [54]:
# looks like it worked, so commit and close
conn.commit()
conn.close()

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.

Code below is exploring messages identified with some of the more sparsely-populated categories.

In [63]:
pd.set_option('display.max_colwidth', None)

In [64]:
display(df[df['offer']==1]['message'][0:20])

75                                                                                                                                                                                                                                     I am a driver, a mechanic ,. I want to help
253                                                                                                                                                                                                                                      How can we help the victims at Les Cayes?
271                                                                                                                                                 I'm the vice president of the association Rafadek in Anse a galets. You can call me so that you could help people in La Gonave
1472                                                                                                                                        Hi i speak english, if it's possibl

In [65]:
df[df['shops']==1]['message'][0:20]

40                                                                                                                                 People from Dal blocked since Wednesday in Carrefour, we having water shortage, food and medical assistance.
167                                                                                                                                     One thing I am asking the money tranfer offices is for them to open so we can get the money sent to us.
237                                                                                             Good evening. Please We need help in Delma 16, number 27. We don't have water, nor food. We don't have tents. Thank you for your understanding!
274                                                                                                                                                                      This did not just happen in Port au Prince it's up in Pot de Paix too.
741     The Office and direction and wal

In [66]:
df[df['tools']==1]['message'][0:20]

84                                                                                                                                                     Please we need help, food and toiletries.
306                         we are in need of food tentes corvers water money. we are in croix des missions/ route butte boyer in the churche mormon an. we are 50 people. pascale saint georges
352                                                                                                     Emergency aid needed at the Centre Flore for Damien on the grounds Pampak of the unibank
761                                                                                              i have a problem talking to people in port au prince, please its talking to people god bless us
889     We've found some fifty houses which suffered mamages with 6 totally down a not enough food to feed people who came did not give us and not enough tools and seeds to plant, thanks a lot
1284                               